# Monte Carlo ES

Line World

In [19]:
# line_world.py
import numpy as np
import random
from collections import namedtuple, defaultdict

class LineWorld:
    def __init__(self, size=5):
        self.size = size
        self.start_state = 0
        self.end_state = size - 1
        self.reset()
        self.action_space = namedtuple('ActionSpace', ['n'])
        self.action_space.n = 2  # Two actions: 0 (left) and 1 (right)
    
    def reset(self):
        self.state = self.start_state
        return self.state
    
    def step(self, action):
        if action == 0:  # Move left
            self.state = max(0, self.state - 1)
        elif action == 1:  # Move right
            self.state = min(self.size - 1, self.state + 1)
        reward = 1 if self.state == self.end_state else 0
        done = self.state == self.end_state
        return self.state, reward, done

    def render(self):
        print(f"State: {self.state}, World: {' '.join(['O' if i == self.state else '-' for i in range(self.size)])}")

def mc_es(env, num_episodes, gamma=1.0):
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    for _ in range(num_episodes):
        episode = []
        state = env.reset()
        action = random.choice(range(env.action_space.n))
        
        while True:
            next_state, reward, done = env.step(action)
            episode.append((state, action, reward))
            if done or len(episode) > 100:
                break
            state = next_state
            action = random.choice(range(env.action_space.n))
        
        G = 0
        for state, action, reward in reversed(episode):
            G = gamma * G + reward
            if (state, action) not in [(x[0], x[1]) for x in episode[:episode.index((state, action, reward))]]:
                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1.0
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]
    
    policy = {state: np.argmax(actions) for state, actions in Q.items()}
    return Q, policy

def visualize_policy(env, policy):
    state = env.reset()
    steps = 0
    while True:
        action = policy[state]
        next_state, reward, done = env.step(action)
        env.render()
        print(f"State: {state}, Action: {action}, Reward: {reward}, Next State: {next_state}")
        state = next_state
        steps += 1
        if done or steps > 100:
            print(f"Episode finished in {steps} steps.")
            break

# Test and visualize LineWorld
if __name__ == "__main__":
    env = LineWorld()
    Q, policy = mc_es(env, num_episodes=1000)
    print("Q-values:", Q)
    print("Policy:", policy)
    visualize_policy(env, policy)


Q-values: defaultdict(<function mc_es.<locals>.<lambda> at 0x110349790>, {3: array([0.99697885, 1.        ]), 2: array([0.99340102, 0.99849398]), 1: array([0.99457259, 0.99562142]), 0: array([0.99100899, 0.99569402])})
Policy: {3: np.int64(1), 2: np.int64(1), 1: np.int64(1), 0: np.int64(1)}
State: 1, World: - O - - -
State: 0, Action: 1, Reward: 0, Next State: 1
State: 2, World: - - O - -
State: 1, Action: 1, Reward: 0, Next State: 2
State: 3, World: - - - O -
State: 2, Action: 1, Reward: 0, Next State: 3
State: 4, World: - - - - O
State: 3, Action: 1, Reward: 1, Next State: 4
Episode finished in 4 steps.


Les Q-values montrent que l'action 1 (se déplacer à droite) a une valeur plus élevée dans chaque état, ce qui est cohérent car le but est d'atteindre l'état final le plus rapidement possible en se déplaçant à droite.
L'agent atteint l'état final en 4 étapes, ce qui montre que la politique apprise est efficace pour ce problème simple.

Grid World

In [20]:
# grid_world.py
import numpy as np
import random
from collections import namedtuple, defaultdict

class GridWorld:
    def __init__(self, width=5, height=5):
        self.width = width
        self.height = height
        self.start_state = (0, 0)
        self.end_state = (width - 1, height - 1)
        self.reset()
        self.action_space = namedtuple('ActionSpace', ['n'])
        self.action_space.n = 4  # Four actions: 0 (up), 1 (down), 2 (left), 3 (right)
    
    def reset(self):
        self.state = self.start_state
        return self.state
    
    def step(self, action):
        x, y = self.state
        if action == 0:  # Move up
            y = max(0, y - 1)
        elif action == 1:  # Move down
            y = min(self.height - 1, y + 1)
        elif action == 2:  # Move left
            x = max(0, x - 1)
        elif action == 3:  # Move right
            x = min(self.width - 1, x + 1)
        self.state = (x, y)
        reward = 1 if self.state == self.end_state else 0
        done = self.state == self.end_state
        return self.state, reward, done

    def render(self):
        grid = [['-' for _ in range(self.width)] for _ in range(self.height)]
        x, y = self.state
        grid[y][x] = 'O'
        for row in grid:
            print(' '.join(row))
        print(f"Current State: {self.state}")

def mc_es(env, num_episodes, gamma=1.0):
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    for _ in range(num_episodes):
        episode = []
        state = env.reset()
        action = random.choice(range(env.action_space.n))
        
        while True:
            next_state, reward, done = env.step(action)
            episode.append((state, action, reward))
            if done or len(episode) > 100:
                break
            state = next_state
            action = random.choice(range(env.action_space.n))
        
        G = 0
        for state, action, reward in reversed(episode):
            G = gamma * G + reward
            if (state, action) not in [(x[0], x[1]) for x in episode[:episode.index((state, action, reward))]]:
                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1.0
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]
    
    policy = {state: np.argmax(actions) for state, actions in Q.items()}
    return Q, policy

def visualize_policy(env, policy):
    state = env.reset()
    steps = 0
    while True:
        action = policy[state]
        next_state, reward, done = env.step(action)
        env.render()
        print(f"State: {state}, Action: {action}, Reward: {reward}, Next State: {next_state}")
        state = next_state
        steps += 1
        if done or steps > 100:
            print(f"Episode finished in {steps} steps.")
            break

# Test and visualize GridWorld
if __name__ == "__main__":
    env = GridWorld()
    Q, policy = mc_es(env, num_episodes=1000)
    print("Q-values:", Q)
    print("Policy:", policy)
    visualize_policy(env, policy)


Q-values: defaultdict(<function mc_es.<locals>.<lambda> at 0x11027f670>, {(4, 3): array([0.47770701, 1.        , 0.55238095, 0.64307692]), (4, 2): array([0.384     , 0.66464646, 0.44043321, 0.44277674]), (4, 1): array([0.33860759, 0.45954693, 0.35538462, 0.37070254]), (4, 0): array([0.34195402, 0.36927224, 0.30699088, 0.31545741]), (3, 0): array([0.34755332, 0.35164835, 0.29006623, 0.33461047]), (2, 0): array([0.3453159 , 0.36231884, 0.34256055, 0.33549784]), (2, 1): array([0.31806616, 0.40767386, 0.32723112, 0.42205323]), (1, 0): array([0.3830156 , 0.40778342, 0.3860262 , 0.39640411]), (0, 0): array([0.44234405, 0.46711328, 0.45626911, 0.44819129]), (1, 1): array([0.35299902, 0.4       , 0.367     , 0.39548577]), (0, 1): array([0.40996503, 0.44298605, 0.4023622 , 0.40448505]), (3, 2): array([0.34751773, 0.5641953 , 0.3777403 , 0.51880878]), (3, 1): array([0.3200569 , 0.42986425, 0.36      , 0.38905325]), (3, 3): array([0.43249428, 0.69347826, 0.4048913 , 0.64233577]), (2, 3): array([0

L'agent atteint l'état final (4, 4) en 8 étapes. Cela montre que la politique apprise permet à l'agent de naviguer efficacement à travers la grille pour atteindre l'état final.

RPS

In [23]:
# two_round_rps.py
import numpy as np
import random
from collections import namedtuple, defaultdict

class TwoRoundRPS:
    def __init__(self):
        self.actions = ['rock', 'paper', 'scissors']
        self.reset()
        self.action_space = namedtuple('ActionSpace', ['n'])
        self.action_space.n = 3  # Three actions: 0 (rock), 1 (paper), 2 (scissors)
    
    def reset(self):
        self.state = []
        self.opponent_moves = [random.choice(self.actions), None]
        return tuple(self.state)
    
    def step(self, action):
        move = self.actions[action]
        self.state.append(move)
        if len(self.state) == 2:
            self.opponent_moves[1] = self.state[0]
            reward = self.get_reward(self.state[1], self.opponent_moves[1])
            done = True
        else:
            reward = 0
            done = False
        return tuple(self.state), reward, done
    
    def get_reward(self, move, opponent_move):
        if move == opponent_move:
            return 0
        elif (move == 'rock' and opponent_move == 'scissors') or \
             (move == 'paper' and opponent_move == 'rock') or \
             (move == 'scissors' and opponent_move == 'paper'):
            return 1
        else:
            return -1

    def render(self):
        print(f"State: {self.state}, Opponent Moves: {self.opponent_moves}")

def mc_es(env, num_episodes, gamma=1.0):
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    for _ in range(num_episodes):
        episode = []
        state = env.reset()
        action = random.choice(range(env.action_space.n))
        
        while True:
            next_state, reward, done = env.step(action)
            episode.append((state, action, reward))
            if done or len(episode) > 100:
                break
            state = next_state
            action = random.choice(range(env.action_space.n))
        
        G = 0
        for state, action, reward in reversed(episode):
            G = gamma * G + reward
            if (state, action) not in [(x[0], x[1]) for x in episode[:episode.index((state, action, reward))]]:
                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1.0
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]
    
    policy = {state: np.argmax(actions) for state, actions in Q.items()}
    return Q, policy

def visualize_policy(env, policy):
    state = env.reset()
    steps = 0
    while True:
        if len(state) == 2:
            break
        action = policy[state]
        next_state, reward, done = env.step(action)
        env.render()
        print(f"State: {state}, Action: {action}, Reward: {reward}, Next State: {next_state}")
        state = next_state
        steps += 1
        if done or steps > 100:
            print(f"Episode finished in {steps} steps.")
            break

# Test and visualize Two Round Rock Paper Scissors
if __name__ == "__main__":
    env = TwoRoundRPS()
    Q, policy = mc_es(env, num_episodes=1000)
    print("Q-values:", Q)
    print("Policy:", policy)
    visualize_policy(env, policy)


Q-values: defaultdict(<function mc_es.<locals>.<lambda> at 0x11054a8b0>, {('rock',): array([ 0.,  1., -1.]), (): array([-0.03625378, -0.04166667,  0.        ]), ('paper',): array([-1.,  0.,  1.]), ('scissors',): array([ 1., -1.,  0.])})
Policy: {('rock',): np.int64(1), (): np.int64(2), ('paper',): np.int64(2), ('scissors',): np.int64(0)}
State: ['scissors'], Opponent Moves: ['scissors', None]
State: (), Action: 2, Reward: 0, Next State: ('scissors',)
State: ['scissors', 'rock'], Opponent Moves: ['scissors', 'scissors']
State: ('scissors',), Action: 0, Reward: 1, Next State: ('scissors', 'rock')
Episode finished in 2 steps.


Les Q-values montrent les récompenses attendues pour chaque action dans différents états. Par exemple, pour l'état ('rock',), l'action 1 (papier) a la récompense la plus élevée. Ici l'agent termine le jeu en 2 étapes

Monty Hall 1

In [24]:
# monty_hall_1.py
import numpy as np
import random
from collections import namedtuple, defaultdict

class MontyHall1:
    def __init__(self):
        self.reset()
        self.action_space = namedtuple('ActionSpace', ['n'])
        self.action_space.n = 2  # Two actions: 0 (stay), 1 (switch)
    
    def reset(self):
        self.doors = ['goat', 'goat', 'car']
        random.shuffle(self.doors)
        self.selected_door = random.randint(0, 2)
        self.revealed_door = self.reveal_door()
        self.state = (self.selected_door, self.revealed_door)
        return self.state
    
    def reveal_door(self):
        available_doors = [i for i in range(3) if i != self.selected_door and self.doors[i] == 'goat']
        return random.choice(available_doors)
    
    def step(self, action):
        if action == 1:  # Switch
            self.selected_door = [i for i in range(3) if i != self.selected_door and i != self.revealed_door][0]
        reward = 1 if self.doors[self.selected_door] == 'car' else 0
        done = True
        return self.state, reward, done

    def render(self):
        print(f"Doors: {self.doors}, Selected Door: {self.selected_door}, Revealed Door: {self.revealed_door}")

def mc_es(env, num_episodes, gamma=1.0):
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    for _ in range(num_episodes):
        episode = []
        state = env.reset()
        action = random.choice(range(env.action_space.n))
        
        while True:
            next_state, reward, done = env.step(action)
            episode.append((state, action, reward))
            if done or len(episode) > 100:
                break
            state = next_state
            action = random.choice(range(env.action_space.n))
        
        G = 0
        for state, action, reward in reversed(episode):
            G = gamma * G + reward
            if (state, action) not in [(x[0], x[1]) for x in episode[:episode.index((state, action, reward))]]:
                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1.0
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]
    
    policy = {state: np.argmax(actions) for state, actions in Q.items()}
    return Q, policy

def visualize_policy(env, policy):
    state = env.reset()
    steps = 0
    while True:
        action = policy[state]
        next_state, reward, done = env.step(action)
        env.render()
        print(f"State: {state}, Action: {action}, Reward: {reward}, Next State: {next_state}")
        state = next_state
        steps += 1
        if done or steps > 100:
            print(f"Episode finished in {steps} steps.")
            break

# Test and visualize Monty Hall Level 1
if __name__ == "__main__":
    env = MontyHall1()
    Q, policy = mc_es(env, num_episodes=1000)
    print("Q-values:", Q)
    print("Policy:", policy)
    visualize_policy(env, policy)


Q-values: defaultdict(<function mc_es.<locals>.<lambda> at 0x11054af70>, {(0, 1): array([0.46835443, 0.625     ]), (2, 0): array([0.30985915, 0.66666667]), (1, 0): array([0.30337079, 0.58510638]), (2, 1): array([0.4025974 , 0.73684211]), (1, 2): array([0.38636364, 0.7       ]), (0, 2): array([0.37647059, 0.63513514])})
Policy: {(0, 1): np.int64(1), (2, 0): np.int64(1), (1, 0): np.int64(1), (2, 1): np.int64(1), (1, 2): np.int64(1), (0, 2): np.int64(1)}
Doors: ['goat', 'car', 'goat'], Selected Door: 1, Revealed Door: 2
State: (0, 2), Action: 1, Reward: 1, Next State: (0, 2)
Episode finished in 1 steps.


Les Q-values montrent que l'action 1 (changer de porte) a une récompense plus élevée, ce qui est attendu dans l'environnement'.
Pour ce qui est de la politique, elle suggère de toujours changer de porte (action 1), ce qui est optimal dans le problème de Monty Hall'. L'agent du coup suit la politique optimale et change de porte, gagnant la voiture en une seule étape.

Monty Hall 2

In [36]:
# monty_hall_2.py
import numpy as np
import random
from collections import namedtuple, defaultdict

class MontyHall2:
    def __init__(self):
        self.reset()
        self.action_space = namedtuple('ActionSpace', ['n'])
        self.action_space.n = 2  # Two actions: 0 (stay), 1 (switch)
    
    def reset(self):
        self.doors = ['goat'] * 4 + ['car']
        random.shuffle(self.doors)
        self.selected_doors = [random.randint(0, 4)]
        self.revealed_doors = self.reveal_doors()
        self.state = (tuple(self.selected_doors), tuple(self.revealed_doors))
        return self.state
    
    def reveal_doors(self):
        available_doors = [i for i in range(5) if i not in self.selected_doors and self.doors[i] == 'goat']
        revealed_doors = random.sample(available_doors, 3)
        return revealed_doors
    
    def step(self, action):
        if action == 1:  # Switch
            remaining_doors = [i for i in range(5) if i not in self.selected_doors and i not in self.revealed_doors]
            self.selected_doors.append(remaining_doors[0])
        reward = 1 if self.doors[self.selected_doors[-1]] == 'car' else 0
        done = True
        self.state = (tuple(self.selected_doors), tuple(self.revealed_doors))
        return self.state, reward, done

    def render(self):
        print(f"Doors: {self.doors}, Selected Doors: {self.selected_doors}, Revealed Doors: {self.revealed_doors}")

def mc_es(env, num_episodes, gamma=1.0):
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    for _ in range(num_episodes):
        episode = []
        state = env.reset()
        action = random.choice(range(env.action_space.n))
        
        while True:
            next_state, reward, done = env.step(action)
            episode.append((state, action, reward))
            if done or len(episode) > 100:
                break
            state = next_state
            action = random.choice(range(env.action_space.n))
        
        G = 0
        for state, action, reward in reversed(episode):
            G = gamma * G + reward
            if (state, action) not in [(x[0], x[1]) for x in episode[:episode.index((state, action, reward))]]:
                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1.0
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]
    
    policy = {state: np.argmax(actions) for state, actions in Q.items()}
    return Q, policy

def visualize_policy(env, policy):
    state = env.reset()
    steps = 0
    while True:
        action = policy[state]
        next_state, reward, done = env.step(action)
        env.render()
        print(f"State: {state}, Action: {action}, Reward: {reward}, Next State: {next_state}")
        state = next_state
        steps += 1
        if done or steps > 100:
            print(f"Episode finished in {steps} steps.")
            break

# Test and visualize Monty Hall Level 2
if __name__ == "__main__":
    env = MontyHall2()
    Q, policy = mc_es(env, num_episodes=1000)
    print("Q-values:", Q)
    print("Policy:", policy)
    visualize_policy(env, policy)


Q-values: defaultdict(<function mc_es.<locals>.<lambda> at 0x11056c040>, {((0,), (4, 3, 2)): array([0.42857143, 1.        ]), ((1,), (2, 4, 3)): array([0., 0.]), ((1,), (3, 2, 0)): array([0.        , 0.83333333]), ((3,), (0, 1, 4)): array([0.25, 0.8 ]), ((0,), (1, 4, 2)): array([0.5, 0.8]), ((2,), (3, 4, 1)): array([0.33333333, 1.        ]), ((3,), (2, 0, 1)): array([0.11111111, 1.        ]), ((2,), (1, 4, 0)): array([0., 1.]), ((2,), (0, 4, 3)): array([0.5       , 0.85714286]), ((4,), (2, 1, 3)): array([0., 1.]), ((3,), (2, 4, 0)): array([0.33333333, 0.85714286]), ((4,), (0, 2, 1)): array([0.25, 1.  ]), ((3,), (4, 2, 0)): array([0.22222222, 0.71428571]), ((2,), (1, 0, 3)): array([0.25, 0.5 ]), ((0,), (2, 1, 4)): array([0., 1.]), ((1,), (2, 0, 4)): array([0.28571429, 1.        ]), ((3,), (1, 2, 4)): array([1.   , 0.875]), ((4,), (2, 0, 1)): array([0., 1.]), ((2,), (0, 3, 4)): array([0.  , 0.75]), ((0,), (1, 2, 3)): array([0.        , 0.83333333]), ((0,), (3, 4, 2)): array([0.25, 0.5 ])

Les Q-values montrent les récompenses attendues pour chaque combinaison de portes sélectionnées et révélées et donc l'agent suit la politique optimale puis gagne la voiture en changeant de porte, ce qui fait qu'il atteint l'état final en une étape aussi.